### Aux function

In [1]:
from pathlib import Path
from typing import Optional, Union

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

In [2]:
def plot_matrix_on_map(
    mat: np.array,
    xlim: tuple = (-180, 180),
    ylim: tuple = (-90, 90),
    label: str = "",
    origin: str = "upper",
    vmin: Optional[Union[int, float]] = None,
    vmax: Optional[Union[int, float]] = None,
    figsize: tuple[float, float] = (14, 6),
    title: str = "",
    colors: str = "viridis_r",
    dpi: int = 100,
    nrow: Optional[int] = None,
    ncol: Optional[int] = None,
    axs: Optional[plt.axis] = None,
) -> tuple[plt.figure, plt.axis]:
    """Plot a matrix over an Earth map. The matrix works as a grid that covers
    the region defined by xlim, ylim.

    Parameters
    ----------
    mat : numpy.array
        Matrix to plot.
    xlim : tuple, optional
        Longitude limits, by default (-180, 180)
    ylim : tuple, optional
        Latitude limits, by default (-90, 90)
    label: str, optional
        Laber of the colorbar. By default, "".
    origin : str, optional
        Set the origin of coordinates. By default, "upper".
    vmin: Optional[Union[int,float]], optional
        Min value of the heatmap. By default, None.
    vmax : Optional[Union[int,float]], optional
        Max value of the heatmap. By default, None.
    figsize : tuple(float, float), optional
        Size of the figure. By default, (14, 6).
    title : str, optional
        Title of the heatmap. By default, "".
    colors : str, optional
        Colors to apply to the heatmap [1]. By default, viridis.
    dpi : int, optional
        Resolution of the figure. By default, 100.
    nrow: Optional[int], optional
        If axs is given, number of rows of the subplot. If not, position at the
        subplot. By default, None.
    ncol: Optional[int], optional
        If axs is given, number of cols of the subplot. If not, position at the
        subplot. By default, None.
    axs: Optional[object], optional
        axs object of a figure to place on it the subplots returned by this
        function. By default, None, so a new figure is created.

    Returns
    -------
    fig, ax : tuple[plt.figure, plt.axis]
        Figure and axis of the subplot that show the input matrix.

    References
    ----------
    .. [1] https://matplotlib.org/stable/tutorials/colors/colormaps.html
    """
    if axs is None:
        num_col = 1 if ncol is None else ncol
        num_row = 1 if nrow is None else nrow
        fig, axs = plt.subplots(num_row, num_col, figsize=figsize, dpi=dpi)
        fig.subplots_adjust(hspace=0.4, wspace=0.2)
    else:
        fig = axs[0][0].figure

    if ncol is None and nrow is None:
        axis = axs
    elif ncol is None:
        axis = axs[nrow - 1]
    elif nrow is None:
        axis = axs[ncol - 1]
    else:
        axis = axs[nrow - 1, ncol - 1]
    
    # specify fignum = 0 so we use current custom axis
    cax = plt.matshow(
        mat,
        fignum=0,
        vmin=vmin,  
        vmax=vmax, 
        extent=xlim + ylim,
        origin=origin,
        cmap=colors,
    )

    axis.set_title(title)
    axis.set_ylim((ylim[0], ylim[1]))
    axis.set_xlim((xlim[0], xlim[1]))
    axis.xaxis.label.set_size(14)
    axis.yaxis.label.set_size(14)
    # Color bar
    im_ratio = (xlim[1] - xlim[0]) / (ylim[1] - ylim[0])
    cbar = fig.colorbar(cax, ax=axis, fraction=0.046 * im_ratio, pad=0.04)
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel(label, rotation=270)

    return fig, axs

# Documentation

https://lagrangian.readthedocs.io/en/latest/scripts/map_of_fle.html 

https://lagrangian.readthedocs.io/en/latest/contents.html#id59 

https://www.aviso.altimetry.fr/en/data/products/value-added-products/fsle-finite-size-lyapunov-exponents/fsle-description.html

In [3]:
import os
import datetime as dt
import pandas as pd

In [4]:
DATES = [dt.datetime(2023, 8, 7), dt.datetime(2023, 8, 15), dt.datetime(2023, 8, 24)]

# Compute fsle from ugos/vgos variables

In [5]:
# computed dates: 2023-08-24, 2023-08-15, 2023-08-7
# map_of_fle duacs.ini 2023_08_24_fsle.nc "2023-08-24" --advection_time 20 --resolution=0.04 --final_separation 0.6 --initial_separation 0.2 --x_min -179.98 --x_max 179.98 --y_min -59.98 --y_max 59.98 --verbose --time_direction backward

In [6]:
# open computed fsle
cmems_ds_list = []
for date in DATES:
    cmems_ds_list.append(xr.open_dataset(f"./{date:%Y_%m_%d}_fsle.nc"))

Add ground mask with AVISO data

In [7]:
# open AVISO data
path_output = Path("/home/SHARED/SATLINK/fsle/")
fsle = sorted(os.listdir(path_output / "aviso"))

aviso_ds_list = []
for date in DATES:
    fname = [p for p in fsle if str(p).startswith(f"{date:%Y_%m_%d}")][0]
    aviso_ds_list.append(xr.open_dataset(path_output / "aviso" / fname))

In [8]:
# add mask ground to computed fsle data and save it
cmems_with_no_ground_ds_list = []
for idx, _ in enumerate(cmems_ds_list):
    cmems_with_ground = cmems_ds_list[idx].copy(deep = True)

    aviso_ground_mask = np.isnan(aviso_ds_list[idx]["fsle_max"].values.squeeze())
    cmems_with_ground["lambda1"].values[aviso_ground_mask.T] = np.nan

    # check number of nans 
    assert(
        np.isnan(cmems_with_ground["lambda1"].values).sum() == 
        aviso_ground_mask.sum() + np.isnan(cmems_ds_list[idx]["lambda1"].values).sum() - len(np.intersect1d(np.isnan(cmems_ds_list[idx]["lambda1"].values.flatten()).nonzero(), aviso_ground_mask.T.flatten().nonzero()))
        )

    # save cmems_ds with ground mask
    # cmems_with_ground.to_netcdf(path=f"./{date:%Y_%m_%d}_fsle_ground.nc")
    cmems_with_no_ground_ds_list.append(cmems_with_ground)

# Print data statistics

In [9]:
# https://stackoverflow.com/questions/40347689/dataframe-describe-suppress-scientific-notation 
for idx, date in enumerate(DATES):
    print(f"\n----------{date:%Y_%m_%d}----------\n")

    print("Computed FSLE statistics (lambda1) ")
    print(pd.DataFrame(cmems_with_no_ground_ds_list[idx]["lambda1"].values.flatten()).describe(percentiles=[.05, .25, .5, .75, .95]).apply(lambda s: s.apply('{0:.5f}'.format)))

    print("\nAVISO statistics")
    print(pd.DataFrame(aviso_ds_list[idx]["fsle_max"].values.flatten()).describe(percentiles=[.05, .25, .5, .75, .95]).apply(lambda s: s.apply('{0:.5f}'.format)))

    aviso_data = np.flipud(aviso_ds_list[idx]["fsle_max"].values.squeeze())
    cmems_lambda1_data= np.flipud(np.transpose(cmems_with_no_ground_ds_list[idx]["lambda1"].values))
    substraction = abs(aviso_data - cmems_lambda1_data)
    print("\n|AVISO - computed FSLE| statistics")
    print(pd.DataFrame(substraction.flatten()).describe(percentiles=[.05, .25, .5, .75, .95]).apply(lambda s: s.apply('{0:.5f}'.format)))


    fig, axs = plt.subplots(1, 1)
    plt.title(f"|AVISO - computed FSLE| histogram ({date:%Y_%m_%d})")
    axs.hist(substraction.flatten(), bins = 500, range = (-0.01, 1))
    fig.savefig(f"./fsle_maps_final/{date:%Y_%m_%d}_substraction_hist.png")
    plt.close(fig)
    


----------2023_08_07----------

Computed FSLE statistics (lambda1) 
                    0
count  19596401.00000
mean         -0.11196
std           0.16297
min          -9.09732
5%           -0.38165
25%          -0.15483
50%          -0.08172
75%           0.00000
95%           0.00000
max           0.00000

AVISO statistics
                    0
count  19596728.00000
mean         -0.06514
std           0.07460
min         -15.16673
5%           -0.18180
25%          -0.07899
50%          -0.04703
75%          -0.02853
95%           0.00000
max           0.00000

|AVISO - computed FSLE| statistics
                    0
count  19596401.00000
mean          0.08799
std           0.12930
min           0.00000
5%            0.00000
25%           0.02802
50%           0.05161
75%           0.10024
95%           0.28746
max          13.82101

----------2023_08_15----------

Computed FSLE statistics (lambda1) 
                    0
count  19596688.00000
mean         -0.11171
std           0.

# AVISO heatmaps 

In [10]:
for idx, date in enumerate(DATES): 
    aviso_data = np.flipud(aviso_ds_list[idx]["fsle_max"].values.squeeze())
    xlim=(aviso_ds_list[idx].lon.min(), aviso_ds_list[idx].lon.max())
    ylim=(aviso_ds_list[idx].lat.min(), aviso_ds_list[idx].lat.max())

    fig1, _ = plot_matrix_on_map(
            aviso_data,
            vmax=None,
            vmin=None,
            xlim = xlim, 
            ylim = ylim,
            figsize=(14, 6),
            title=f"AVISO heatmap ({date:%Y_%m_%d})",
        )
    fig1.savefig(f"./fsle_maps_final/{date:%Y_%m_%d}_AVISO.png")
    plt.close(fig1)

    # clipped computed data to [-1, 0]
    clipped_data = np.clip(aviso_data, -1, 0)
    fig2, _ = plot_matrix_on_map(
            clipped_data,
            vmax=None,
            vmin=None,
            xlim = xlim, 
            ylim = ylim,
            figsize=(14, 6),
            title=f"AVISO heatmap (clipped) ({date:%Y_%m_%d})",
        )
    fig2.savefig(f"./fsle_maps_final/{date:%Y_%m_%d}_AVISO_clipped.png")
    plt.close(fig2)

# Computed FSLE heatmaps

In [11]:
for idx, date in enumerate(DATES): 
    cmems_no_ground_data= np.flipud(np.transpose(cmems_with_no_ground_ds_list[idx]["lambda1"].values))
    xlim=(cmems_with_no_ground_ds_list[idx].lon.min(), cmems_with_no_ground_ds_list[idx].lon.max())
    ylim=(cmems_with_no_ground_ds_list[idx].lat.min(), cmems_with_no_ground_ds_list[idx].lat.max())
    
    fig1, _ = plot_matrix_on_map(
            cmems_no_ground_data,
            vmax=None,
            vmin=None,
            xlim = xlim, 
            ylim = ylim,
            figsize=(14, 6),
            title=f"Computed FSLE heatmap ({date:%Y_%m_%d})",
        )
    fig1.savefig(f"./fsle_maps_final/{date:%Y_%m_%d}_computed_FSLE.png")
    plt.close(fig1)


    # rescaled computed data to [-1, 0]
    rescaled_data = ((cmems_no_ground_data - np.nanmin(cmems_no_ground_data)) / (np.nanmax(cmems_no_ground_data) - np.nanmin(cmems_no_ground_data))) - 1
    fig2, _ = plot_matrix_on_map(
            rescaled_data,
            vmax=None,
            vmin=None,
            xlim = xlim, 
            ylim = ylim,
            figsize=(14, 6),
            title=f"Computed FSLE heatmap (rescaled) ({date:%Y_%m_%d})",
        )
    fig2.savefig(f"./fsle_maps_final/{date:%Y_%m_%d}_computed_FSLE_rescaled.png")
    plt.close(fig2)


    # clipped computed data to [-1, 0]
    clipped_data = np.clip(cmems_no_ground_data, -1, 0)
    fig3, _ = plot_matrix_on_map(
            clipped_data,
            vmax=None,
            vmin=None,
            xlim = xlim, 
            ylim = ylim,
            figsize=(14, 6),
            title=f"Computed FSLE heatmap (clipped) ({date:%Y_%m_%d})",
        )
    fig3.savefig(f"./fsle_maps_final/{date:%Y_%m_%d}_computed_FSLE_clipped.png")
    plt.close(fig3)

# |AVISO - computed FSLE| heatmaps

In [12]:
for idx, date in enumerate(DATES): 
    aviso_data = np.flipud(aviso_ds_list[idx]["fsle_max"].values.squeeze())
    cmems_lambda1_data= np.flipud(np.transpose(cmems_with_no_ground_ds_list[idx]["lambda1"].values))
    substraction = abs(aviso_data - cmems_lambda1_data)
    
    xlim=(aviso_ds_list[idx].lon.min(), aviso_ds_list[idx].lon.max())
    ylim=(aviso_ds_list[idx].lat.min(), aviso_ds_list[idx].lat.max())

    fig1, _ = plot_matrix_on_map(
            substraction,
            vmax=None,
            vmin=None,
            xlim = xlim, 
            ylim = ylim,
            figsize=(14, 6),
            colors = "viridis",
            title=f"|AVISO - FSLE| heatmap ({date:%Y_%m_%d})",
        )
    fig1.savefig(f"./fsle_maps_final/{date:%Y_%m_%d}_substraction.png")
    plt.close(fig1)

    # clipped computed data to [0, 1]
    substraction_clipped = np.clip(substraction, 0, 1)
    fig3, _ = plot_matrix_on_map(
            substraction_clipped,
            vmax=None,
            vmin=None,
            xlim = xlim, 
            ylim = ylim,
            figsize=(14, 6),
            colors = "viridis",
            title=f"|AVISO - FSLE| heatmap (clipped) ({date:%Y_%m_%d})",
        )
    fig3.savefig(f"./fsle_maps_final/{date:%Y_%m_%d}_substraction_clipped.png")
    plt.close(fig3)